### Preamble

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pandas as pd
%cd /home/jeroen/repos/traffic-scheduling/single

/home/jeroen/repos/traffic-scheduling/single


In [10]:
# enable caching of intermediate results
import dill as pickle # (to store anonymous functions)
from joblib import Memory
memory = Memory('cache', verbose=0)

## Specify experiments

Experiment ingredients: distributions and scheduling models (i.e., procedures that train/eval).

In [11]:
from instances import bimodal_exponential

# some fixed gap distributions
gap_distributions = {
    # 'low', 'med' and 'high' have same arrival intensity,
    # computed using s2 = (mu - p*s1) / (1 - p)
    'high': bimodal_exponential(p=0.5, s1=0.1, s2=10.00),
    'med':  bimodal_exponential(p=0.3, s1=0.1, s2= 7.17),
    'low':  bimodal_exponential(p=0.1, s1=0.1, s2= 5.6),
}

# from imitation import ImitationLearning
from ppo import PpoRl
from threshold import ThresholdGridSearch, ThresholdBrentSearch

import numpy as np

# model classes, possibly parameterized
models = {
    'Grid':    ThresholdGridSearch(tau_range=np.linspace(0, 2, 10)),
    # 'Brent':   ThresholdBrentSearch(maxiter=10),
    # 'PPO1':                 PpoRl(steps=100, verbose=0),
    # 'PPO2':                 PpoRl(steps=500, verbose=0),
    # 'Imitation':            ImitationLearning(),
    # 'Imitation100':         Imitation(train_size=100), # ...or something along those lines
    # 'Single step REINFOCE': SingleStepRl,
    # 'Episodic REINFORCE':   EpisodeRl,
}

Specify experiments by combining the above "ingredients". When the train distribution differs from the eval distribution, we are effectively measuring how well the model generalizes.

In [12]:
# global number of test instances used in model evaluation
N_test = 100

# definition of experiments (measure generalization by providing different test (n, F) pair)
exps = [
    ([10, 10], 'low', 'Grid', [10, 10], 'low'),
    ([10, 10], 'low', 'Grid', [10, 10], 'low'),
]
experiments = pd.DataFrame(exps, columns=['n_train', 'F_train', 'model_spec', 'n_test', 'F_test'])

# next, we "compile" these textual experiment specifications
# - add explicit generator function based on the textual specification
# - instantiate each model class (optionally with parameters)
from instances import generate_instance
for i, row in experiments.iterrows():
    for m in ['train', 'test']:
        n = row[f'n_{m}']; F = row[f'F_{m}']
        # when F is not a list, broadcast according to number of routes
        if isinstance(F, str): 
            F = [F]*len(n)
            row[f'F_{m}'] = F # write broadcasted list back
        F = [gap_distributions[f] for f in F]
        experiments.loc[i, f'gen_{m}'] = lambda: generate_instance(F, n=n)

# look up model by name, put in pandas table
experiments['model'] = experiments.apply(lambda row: models[row['model_spec']], axis=1)

experiments.drop(columns=['gen_train', 'gen_test', 'model'])

,n_train,F_train,model_spec,n_test,F_test
0,"[10, 10]","[low, low]",Grid,"[10, 10]","[low, low]"
1,"[10, 10]","[low, low]",Grid,"[10, 10]","[low, low]"


## Run all experiments

WATCH OUT: timelimit is currently set very low for development!

In [13]:
from exact import solve
import time
import numpy as np

@memory.cache
def gen_test_data_and_optimal_solutions(gen):
    print('Generating test data')
    instances = [gen() for _ in range(N_test)]
    solutions = []

    start_time = time.time()
    for s in instances:
        solution = solve(s)
        # calculate the "delay" objective by subtracting sum of earliest crossing times
        solution['obj_delay'] = solution['obj'] - np.sum(s['release'])
        solutions.append(solution)
    time_opt = time.time() - start_time
    return instances, solutions, time_opt

In [14]:
for i, exp in experiments.iterrows():
    print(f"Experiment {i}: {exp['n_train']}, {exp['F_train']}"
          f" -> {exp['model_spec']} -> {exp['n_test']}, {exp['F_test']}")

    # create test instances and solve them to optimality (CACHED)
    test_instances, solutions, time_opt = gen_test_data_and_optimal_solutions(exp['gen_test'])

    # ref to instantiated model
    model = exp['model']

    # train model
    start_time = time.time()
    model.train(exp['gen_train'])
    time_train = time.time() - start_time

    # evaluate model
    start_time = time.time()
    # NOTE: also accept std?
    obj = model.eval(exp['gen_test'])
    time_test = time.time() - start_time
    
    # compute:
    # - average optimal objective (TODO: scale)
    # - average model objective (TODO: scale)
    # - optimality gap
    opt = sum([t['obj_delay'] for t in solutions]) / len(solutions)
    experiments.loc[i, 'opt'] = opt
    experiments.loc[i, 'obj'] = obj
    experiments.loc[i, 'gap'] = 100 * ((obj / opt) - 1)
    experiments.loc[i, 'time_opt'] = time_opt
    experiments.loc[i, 'time_train']  = time_train
    experiments.loc[i, 'time_test'] = time_test

from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Done at: {now}")
import os
os.system('notify-send "traffic-scheduling" "Experiments done"')

Experiment 0: [10, 10], ['low', 'low'] -> Grid -> [10, 10], ['low', 'low']
Generating test data


TypeError: 'SingleInstance' object is not subscriptable

## Generate results table

In [ ]:
results = experiments.drop(columns=['gen_train', 'gen_test', 'model'])

# convert lists of strings like ['low', 'low'] into actual string '[low, low]'
for i, row in results.iterrows():
    for col in ['n_train', 'F_train', 'n_test', 'F_test']:
        results.loc[i, col] = r'\texttt{[' + ', '.join(str(i) for i in row[col]) + r']}'

# general precaution: replace underscores with spaces to avoid latex errors
results.columns = results.columns.str.replace('_', ' ')

results

,n train,F train,model spec,n test,F test,opt,obj,gap,time opt,time train,time test
0,"\texttt{[10, 10]}","\texttt{[low, low]}",Grid,"\texttt{[10, 10]}","\texttt{[low, low]}",6.336778,2.990904,-52.800872,0.855222,2.051633,0.000742
1,"\texttt{[10, 10]}","\texttt{[low, low]}",Grid,"\texttt{[10, 10]}","\texttt{[low, low]}",6.336778,8.088388,27.641952,0.855222,2.047788,0.000587


In [ ]:
# pivot
results = results.pivot_table(
    index=['n train', 'F train'],   # rows
    columns='model spec',           # columns
    values=['opt', 'time opt', 'obj', 'gap', 'time train', 'time test']  # metrics per method
)
results

,,gap,obj,opt,time opt,time test,time train
,model spec,Grid,Grid,Grid,Grid,Grid,Grid
n train,F train,,,,,,
"\texttt{[10, 10]}","\texttt{[low, low]}",-12.57946,5.539646,6.336778,0.855222,0.000664,2.049711


In [ ]:
# swap levels of MultiIndex
results.columns = results.columns.swaplevel(0, 1)
# sort columns by model first
results = results.sort_index(axis=1, level=0)
results

model spec                                 Grid                                \
                                            gap       obj       opt  time opt   
n train           F train                                                       
\texttt{[10, 10]} \texttt{[low, low]} -12.57946  5.539646  6.336778  0.855222   

model spec                                                  
                                      time test time train  
n train           F train                                   
\texttt{[10, 10]} \texttt{[low, low]}  0.000664   2.049711

In [ ]:
# collapse the 'opt' and 'time opt' columns into new top-level column
# (because these are guaranteed to be the same for every method, by construction; and caching)
top_cols_to_collapse = results.columns.get_level_values(0).unique()
sub_cols_to_collapse = ['opt', 'time opt']

# take first column per second-level (safe because values are equal)
collapsed = results[top_cols_to_collapse].T.groupby(level=1).first().T
collapsed = collapsed[sub_cols_to_collapse]

# assign a new top-level name
collapsed.columns = pd.MultiIndex.from_product([["MILP"], collapsed.columns])

# drop original columns and concatenate new column back to the original df
results = pd.concat([results.drop(columns=sub_cols_to_collapse, level=1), collapsed], axis=1)

In [ ]:
# get current top-level names
top_levels = results.columns.get_level_values(0).unique()
# put MILP column first
new_order = ["MILP"] + [col for col in top_levels if col != "MILP"]
results = results.reindex(columns=new_order, level=0)

# order sub-level names
sub_level_order = ['obj', 'gap', 'time train', 'time test', 'opt', 'time opt']
results = results.reindex(columns=sub_level_order, level=1)

# rename the top and bottom level
results.columns.names = ["model", "measurement"]

results

model                                      MILP                Grid            \
measurement                                 opt  time opt       obj       gap   
n train           F train                                                       
\texttt{[10, 10]} \texttt{[low, low]}  6.336778  0.855222  5.539646 -12.57946   

model                                                       
measurement                           time train time test  
n train           F train                                   
\texttt{[10, 10]} \texttt{[low, low]}   2.049711  0.000664

In [ ]:
# further index and column renaming to taste
results.rename(columns={
    'time opt':  r'$T$',
    'time train': r'$T_\text{train}$',
    'time test':  r'$T_\text{eval}$',
    'n train': r'$n_r$',
    'F train': r'$F_\text{train}',
}, inplace=True)
results.index.rename([r'$n_r$', r'$F_\text{train}$'], inplace=True)

In [ ]:
# convert to gaps strings
cols_x = results.columns.get_level_values(1) == 'gap'
results = results.astype({ col: str for col in results.columns[cols_x] })

def bold_min_sublevel(row):
    new_row = row.copy()
    cols_x = row.index.get_level_values(1) == 'gap'
    # find min among these columns
    min_val = row[cols_x].astype(float).min()
    # apply bold only to min in that sublevel
    new_row[cols_x] = [f"\\textbf{{{float(v):.2f}}}\\%" if float(v)==min_val else f"{float(v):.2f}\\%" 
                       for v in row[cols_x]]
    return new_row

# hightlight min gap, in each row
results = results.apply(bold_min_sublevel, axis=1)

In [ ]:
from single.util import format_duration

# 2-leveled columns, so apply formatting at every columns,
# e.g. (PPO1, gap), (PPO2, gap), separately
time_cols = [r'$T$', r'$T_\text{train}$', r'$T_\text{eval}$']
time_cols = results.columns[results.columns.isin(time_cols, 1)]
formatters = { col: format_duration for col in time_cols }

# latex \begin{table}{...} format
column_format = 'cc|cc|cccc|cccc'
latex_table = results.to_latex(multirow=True, column_format=column_format, multicolumn_format='c',
                               escape=False, float_format="%.2f", formatters=formatters,
                               caption="Performance Table")
print(latex_table)

\begin{table}
\caption{Performance Table}
\begin{tabular}{cc|cc|cccc|cccc}
\toprule
 & model & \multicolumn{2}{c}{MILP} & \multicolumn{4}{c}{Grid} \\
 & measurement & opt & $T$ & obj & gap & $T_\text{train}$ & $T_\text{eval}$ \\
$n_r$ & $F_\text{train}$ &  &  &  &  &  &  \\
\midrule
\texttt{[10, 10]} & \texttt{[low, low]} & 6.34 & (1s) & 5.54 & \textbf{-12.58}\% & (2s) & (0s) \\
\cline{1-8}
\bottomrule
\end{tabular}
\end{table}



In [ ]:
import re

# some further automated formatting (just search/replace)
# 1. remove all \cline
latex_table = re.sub(r'\\cline\{.*?\}\s*\n?', '', latex_table)
# 2. remove \toprule
latex_table = re.sub(r'\\toprule\s*\n?', '', latex_table)
# 3. replace \bottomrule with \midrule
latex_table = re.sub(r'\\bottomrule\s*\n?', '\\\\midrule\n', latex_table)
# 4. TODO: extract tabular
# latex_table = re.sub()
match = re.search(r"\\begin\{tabular.*?\\end\{tabular\}", latex_table, re.DOTALL)
latex_table = match.group(0)

# write complete latex template for preview
# (with timestamp)
from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
date_line = f"\\noindent\\textit{{Compiled on: {now}}}\\\\\n"
latex = f"""
\\documentclass{{article}}
\\usepackage{{booktabs}}
\\usepackage{{multirow}}
\\usepackage{{siunitx}}
\\usepackage[skip=10pt]{{caption}}
\\usepackage[margin=1cm]{{geometry}}
\\begin{{document}}
\\begin{{table}}
{latex_table}
\\end{{table}}
{date_line}
\\end{{document}}"""

file_prefix = 'results'

with open(file_prefix + '.preview.tex', 'w') as f:
    f.write(latex)
!tectonic {file_prefix + '.preview.tex'}

Running TeX ...
Rerunning TeX because "results.preview.aux" changed ...
Running xdvipdfmx ...
Writing `results.preview.pdf` (16.83 KiB)
Skipped writing 1 intermediate files (use --keep-intermediates to keep them)


In [ ]:
# write only table, for direct use in report
with open(file_prefix + '.tex', 'w') as f:
    f.write(latex_table)

## Additional loss plots and training curves

In [ ]:
from stable_baselines3.common.results_plotter import load_results

df = load_results("logs/")
print(df.head())
import matplotlib.pyplot as plt

window = 50
df["r_smooth"] = df["r"].rolling(window, min_periods=1).mean()

plt.figure(figsize=(8,4))
plt.plot(df["t"], df["r"], alpha=0.3, label="Episodic reward")
plt.plot(df["t"], df["r_smooth"], linewidth=2, label=f"Smoothed ({window})")
plt.xlabel("Time (s)")
plt.ylabel("Episode reward")
plt.legend()
plt.grid(True)
plt.show()

LoadMonitorResultsError: No monitor files of the form *monitor.csv found in logs/